In [1]:
import obspy
from obspy.geodetics.flinnengdahl import FlinnEngdahl
import pandas as pd
import numpy as np
import datetime 
import re
import os

from selenium import webdriver as wd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen
import requests
import time

In [7]:
# hypo.phase 자동화 (반올림)

# sac 폴더 내 파일 중 '*Z.sac' 로 끝나는 파일명 리스트화 -> sort(key= )로 station명 3글자인 관측데이터부터 우선 정렬 (알파벳순 자동)
# '../sac/' 등 코드 내 경로명은 본인 환경에 맞게 수정하세요
file = os.listdir('../dist100/dist100_station2')
array = list(filter(lambda x: re.match('.*Z.sac$', x), file))
array.sort(key=lambda x : len(x))

dic = {}
unit_array = []
for i in array:
    sta, ks, cha = i.split('.')[0], i.split('.')[1], i.split('.')[2]
    if cha == 'ELZ':
        dic[sta] = cha
    elif cha == 'HGZ':
        dic[sta] = cha
    elif cha == 'HHZ' :
        dic[sta] = cha
    else :
        pass
    
for key, value in dic.items():    
    unit = '{0}.KS.{1}.sac'.format(key, value)
    unit_array.append(unit)
    
# 위에서 만든 리스트에서 필요한 데이터만 추출해서 hypo.phase에 한 줄씩 저장하는 코드
with open('hypo.phase', 'a') as f:
    for i in unit_array:
        st = obspy.read('../dist100/dist100_station2/' + i)
        # 터미널에서 saclst로 필요한 데이터 가져오는 코드와 동일
        # saclst와 다르게 month & day가 아닌 julday로 반환하는 것 주의
        tnm, ka, year, jday, hour, minute, sec, msec, a, t0, kt0 = st[0].stats.sac.kstnm, st[0].stats.sac.ka, st[0].stats.sac.nzyear, st[0].stats.sac.nzjday, st[0].stats.sac.nzhour, st[0].stats.sac.nzmin, st[0].stats.sac.nzsec, st[0].stats.sac.nzmsec, st[0].stats.sac.a, st[0].stats.sac.t0, st[0].stats.sac.kt0
        # HHZ > HGZ > ELZ 순의 우선순위 적용
                
        # 정확한 시간 계산 위해 obspy.UTCDateTime.timestamp로 nanoseconds 형식의 P파 도달시간, S파 도달시간, PS시 추출
        # 정확한 계산 위해 btime값만큼 eventtime에서 빼줌 
        timestamp = obspy.UTCDateTime(year=year, julday=jday, hour=hour, minute=minute, second=sec, microsecond=msec).timestamp - st[0].stats.sac.b
        p_timestamp = timestamp + a
        s_timestamp = timestamp + t0
        ps = s_timestamp - p_timestamp
        # 시간 계산 이후 hypo.phase에 들어갈 포맷으로 변경한 P파 도달시간을 ptime 변수에 입력
        ptime_org = obspy.UTCDateTime(p_timestamp)
        ptime = obspy.UTCDateTime(p_timestamp).strftime('%y%m%d%H%M%S.%f')
        ptime = '{:.2f}'.format(np.float64(ptime))
        # S파 도달시간은 P파 도달시간을 기준으로 하기 때문에 datetime으로 계산하지 않고 먼저 P파 도달시간의 second, microsecond를 문자형으로 바꿔준 후 PS시를 더해주어 60초가 넘어도 1분으로 넘어가지 않고 그대로 입력되게끔 설정해줌
        # 어차피 hypo.phase에 집어넣으면 텍스트에 불과하기 때문에 ptime, stime 변수 string type으로 집어넣어도 무관
        ptime_str = str(ptime_org.second) + '.' + str(ptime_org.microsecond)
        stime = str(float(ptime_str) + ps)
        # 위에서 구한 stime이 0초 이상 10초 미만 일 시 python에서는 '01:11'과 같이 출력되지 않고 '1:11'로 출력되기 때문에 hypo.phase가 인식할 수 있게끔 이러한 데이터들만 앞에 0을 수동으로 붙여줌 
        if re.match(r'^[1-9][.]', stime):
            stime = '{:.2f}'.format(np.float64(stime), 2)
            stime = '0' + str(stime)
        else:
            stime = '{:.2f}'.format(np.float64(stime), 2)
        # station명이 3글자일경우 뒤에 띄어쓰기를 무조건 해주어야 hypo.phase가 인식될 수 있음 -> 조건문 만들어서 예외 경우 만듬
        if len(tnm) == 3:
            line = '{0} {1} {2}       {3}{4}'.format(tnm, ka, ptime, stime, kt0)
        else:
            line = '{0}{1} {2}       {3}{4}'.format(tnm, ka, ptime, stime, kt0)
        print(tnm)
        # 위에서 만든 formatted line을 hypo.phase에 입력해주고 반복문 루프가 돌 때마다 줄 넘김까지 적용
        f.write(line + '\n')

BAU
MND
BLGA
BON2
BUYB
CDSA
CE2A
CGYA
CHOA
CHRB
CIGB
CJDB
CLSA
CPR2
DAYA
DGHA
DUSB
EURB
GICA
GIJA
GKSA
GMNA
GOCB
GUMA
GWYB
HACA
HAMB
HCNA
HONA
HSJB
HWSA
IKSA
IMSB
JAGA
JASA
JEU2
JGNA
JINA
JLSA
KCH2
KMSB
KWJ2
KWJU
LIWA
MALA
MNDB
MSMB
NACA
NAWB
NOSA
OKCB
OSBA
PHDA
PUAA
SACA
SAJB
SCHA
SECA
SEJA
SENA
SMWA
SUCA
TAIA
TEJ2
WAJB
WICA
YALB
YDGA
YGAA
YUGA


In [3]:
'''
# hypo.phase 자동화 (소숫점 아래 버림)

file = os.listdir('../sac') 
array = list(filter(lambda x: re.match('.*Z.sac$', x), file))
array.sort(key=lambda x : len(x))

with open('hypo.phase', 'a') as f:
    for i in array:
        st = obspy.read('../sac/' + i)
        
        tnm, ka, year, jday, hour, min, sec, msec, a, t0, kt0 = st[0].stats.sac.kstnm, st[0].stats.sac.ka, st[0].stats.sac.nzyear, st[0].stats.sac.nzjday, st[0].stats.sac.nzhour, st[0].stats.sac.nzmin, st[0].stats.sac.nzsec, st[0].stats.sac.nzmsec, st[0].stats.sac.a, st[0].stats.sac.t0, st[0].stats.sac.kt0
        timestamp = obspy.UTCDateTime(year=year, julday=jday, hour=hour, minute=min, second=sec, microsecond=msec).timestamp
        p_timestamp = timestamp + a
        s_timestamp = timestamp + t0
        ps = s_timestamp - p_timestamp
        ptime_org = obspy.UTCDateTime(p_timestamp)
        ptime = str(obspy.UTCDateTime(p_timestamp).strftime('%y%m%d%H%M%S.%f'))[:15]
        
        ptime_str = str(ptime_org.second) + '.' + str(ptime_org.microsecond)
        stime = str(float(ptime_str) + ps)
        if re.match(r'^[1-9][.]', stime):
            stime = stime[:4]
            stime = '0' + stime
        else:
            stime = stime[:5]
        if len(tnm) == 3:
            line = '{0} {1} {2}       {3}{4}'.format(tnm, ka, ptime, stime, kt0)
        else:
            line = '{0}{1} {2}       {3}{4}'.format(tnm, ka, ptime, stime, kt0)
        f.write(line + '\n')
'''

"\n# hypo.phase 자동화 (소숫점 아래 버림)\n\nfile = os.listdir('../sac') \narray = list(filter(lambda x: re.match('.*Z.sac$', x), file))\narray.sort(key=lambda x : len(x))\n\nwith open('hypo.phase', 'a') as f:\n    for i in array:\n        st = obspy.read('../sac/' + i)\n        \n        tnm, ka, year, jday, hour, min, sec, msec, a, t0, kt0 = st[0].stats.sac.kstnm, st[0].stats.sac.ka, st[0].stats.sac.nzyear, st[0].stats.sac.nzjday, st[0].stats.sac.nzhour, st[0].stats.sac.nzmin, st[0].stats.sac.nzsec, st[0].stats.sac.nzmsec, st[0].stats.sac.a, st[0].stats.sac.t0, st[0].stats.sac.kt0\n        timestamp = obspy.UTCDateTime(year=year, julday=jday, hour=hour, minute=min, second=sec, microsecond=msec).timestamp\n        p_timestamp = timestamp + a\n        s_timestamp = timestamp + t0\n        ps = s_timestamp - p_timestamp\n        ptime_org = obspy.UTCDateTime(p_timestamp)\n        ptime = str(obspy.UTCDateTime(p_timestamp).strftime('%y%m%d%H%M%S.%f'))[:15]\n        \n        ptime_str = str(ptime_

---

In [5]:
# NECIS 해발고도 자동으로 긁어오기 (코드 수정 필요, 작동은 하는데 웬만하면 실행 ㄴㄴ)

# Selenium moduie 및 경로 내에 chromedriver.exe 존재해야 합니다 (https://wikidocs.net/91474)
# 느리면 30분 정도 소요, 크롤링 진행 중에 chrome 건들지 말기
# 마지막에 오류나는거 당연한거니까 당황 ㄴㄴ

series = pd.Series()

try: 

    options = wd.ChromeOptions()
    #options.add_argument("headless")
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    #options.add_argument('--start-maximized')
    options.add_argument('--blink-settings=imagesEnabled=false')
    driver = wd.Chrome(executable_path = 'chromedriver.exe', options = options)

    
    # 에러 방지차 일부러 시간 지연
    driver.implicitly_wait(3)

    # NECIS 자동 로그인
    driver.get('https://necis.kma.go.kr/necis-dbf/user/common/userLoginNewForm.do')
    element_id = driver.find_element(By.ID, 'email')
    element_id.send_keys('본인 NECIS 이메일 입력하세요')
    element_pwd = driver.find_element(By.ID,'pPasswd')
    element_pwd.send_keys('본인 NECIS 비밀번호 입력하세요')
    time.sleep(3)
    driver.find_element(By.CLASS_NAME, 'btn_login').click()
    time.sleep(5)

    # 지진관측소 페이지 이동 및 데이터 추출
    driver.get('https://necis.kma.go.kr/necis-dbf/user/ob/earthquakeObservatoryListPage.do?fromFlag=start')
    
    for h in range(1, 11):
        sta_button = driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/div[2]/div[2]/table/tbody/tr['+ str(h) +']/td[3]').click()
        driver.implicitly_wait(5)
        sta = driver.find_element(By.ID, "spotSta").get_attribute('value')
        elev = driver.find_element(By.ID, "spotElev").get_attribute('value')
        series[str(sta)] = elev
        print(sta)
        driver.back()
        
    for i in range(2, 12):
        driver.switch_to.window(driver.window_handles[0]) 
        time.sleep(3)
        for j in range(10):
            driver.execute_script('window.open("https://necis.kma.go.kr/necis-dbf/user/ob/earthquakeObservatoryListPage.do?fromFlag=start");')
            driver.switch_to.window(driver.window_handles[-1]) 
            time.sleep(3)
            driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/div[2]/div[2]/div[2]/div/a[' + str(i) + ']').click()
        
        tabs = driver.window_handles
        for k in range(1, 11):
            driver.switch_to.window(tabs[k])
            sta_button = driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/div[2]/div[2]/table/tbody/tr['+ str(k) +']/td[3]').click()
            driver.implicitly_wait(5)
            sta = driver.find_element(By.ID, "spotSta").get_attribute('value')
            elev = driver.find_element(By.ID, "spotElev").get_attribute('value')
            series[str(sta)] = elev
            print(sta)
            driver.close()
    
    for l in range(3, 13):
        driver.switch_to.window(driver.window_handles[0]) 
        time.sleep(3)
        for m in range(10):
            driver.execute_script('window.open("https://necis.kma.go.kr/necis-dbf/user/ob/earthquakeObservatoryListPage.do?fromFlag=start");')
            driver.switch_to.window(driver.window_handles[-1])
            time.sleep(3)
            driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/div[2]/div[2]/div[2]/div/a[11]').click()
            time.sleep(3)
            driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/div[2]/div[2]/div[2]/div/a[' + str(l) + ']').click()
        
        tabs = driver.window_handles
        for n in range(1, 11):
            driver.switch_to.window(tabs[n])
            sta_button = driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/div[2]/div[2]/table/tbody/tr['+ str(n) +']/td[3]').click()
            driver.implicitly_wait(5)
            sta = driver.find_element(By.ID, "spotSta").get_attribute('value')
            elev = driver.find_element(By.ID, "spotElev").get_attribute('value')
            series[str(sta)] = elev
            print(sta)
            driver.close()       
 
    for o in range(3, 13):
        driver.switch_to.window(driver.window_handles[0]) 
        time.sleep(3)
        for p in range(10):
            driver.execute_script('window.open("https://necis.kma.go.kr/necis-dbf/user/ob/earthquakeObservatoryListPage.do?fromFlag=start");')
            driver.switch_to.window(driver.window_handles[-1])
            time.sleep(3)
            driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/div[2]/div[2]/div[2]/div/a[11]').click()
            time.sleep(3)
            driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/div[2]/div[2]/div[2]/div/a[12]').click()
            time.sleep(3)
            driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/div[2]/div[2]/div[2]/div/a[' + str(o) + ']').click()
        
        tabs = driver.window_handles
        for q in range(1, 11):
            driver.switch_to.window(tabs[q])
            sta_button = driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/div[2]/div[2]/table/tbody/tr['+ str(q) +']/td[3]').click()
            driver.implicitly_wait(5)
            sta = driver.find_element(By.ID, "spotSta").get_attribute('value')
            elev = driver.find_element(By.ID, "spotElev").get_attribute('value')
            series[str(sta)] = elev
            print(sta)
            driver.close()       

except Exception:
    raise

finally:
    if driver is not None:
        driver.quit()

C:\Users\neimb\AppData\Local\Temp/ipykernel_18152/1608184008.py:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  series = pd.Series()
C:\Users\neimb\AppData\Local\Temp/ipykernel_18152/1608184008.py:15: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = wd.Chrome(executable_path = 'chromedriver.exe', options = options)


ADOA
AGSA
AMD
ANDB
ANHA
ANMA
ASNA
BAR2
BAU
BGDB
BKWA
BLGA
BLLA
BOGA
BOHB
BON2
BOSB
BSAA
BURB
BUS3
BUSA
BUYB
CDSA
CE2A
CEA2
CEBA
CGAA
CGDA
CGIA
CGPA
CGUB
CGWB
CGYA
CHDA
CHIA
CHJ3
CHOA
CHRB
CHYB
CIGB
CJD
CJDB
CLSA
CPR2
CSDB
CSOA
CWO2
DAGA
DAYA
DDCA
DDCB
DEI2
DGHA
DGJA
DGLA
DKSA
DNBA
DNYA
DUSB
ECDB
EURB
EUSB
GACA
GAGA
GAPB
GBI2
GDDB
GEJA
GEJB
GGDA
GGDB
GGGA
GGSA
GGTA
GH2B
GICA
GIGA
GIJA
GKSA
GLCA
GLSA
GMDB
GMHB
GMNA
GMPB
GOBA
GOCB
GODA
GOSF
GSGA
GSNA
GUJA
GUMA
GUWB
GWLA
GWYB
HA2B
HACA
HAIA
HAMB
HANB
HAWA
HAWB
HCNA
HEDA
HESA
HGDB
HGSA
HMGA
HMPA
HOCA
HONA
HSJB
HTDA
HWCA
HWDB
HWSA
HYDA
ICN2
IJA2
IJAA
IJBA
IJDB
IKSA
IMSB
IMWB
INCA
INJA
IPTA
ISHB
IYIB
IYPA
JAEA
JAEB
JAGA
JAHA
JASA
JCUA
JDGB
JDO2
JECB
JEDA
JEJB
JESA
JEU2
JGJA
JGNA
JHCB
JINA
JJBA
JJU2
JKJA
JLSA
JMJ2
JNHA
JNPA
JNUA
JNYA
JODB
JOGB
JUCA
JURA
JWJB
KAWA
KCH2
KESB
KH2B
KKDA
KMSB
KOSB
KUJA
KWJ2
KWJU
LIWA
LMGA
MALA
MANA
MGY2
MJDB
MLGA
MMD
MND
MNDB
MOGA
MOPB
MSMB
MSNA
MUS2
NACA
NAHA
NAJA
NAMB
NAWB
NCNA
NGJA
NHPA
NHSB
NJDA
NLDA
NOSA
NPGA


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[1]/div[3]/div[2]/div[2]/table/tbody/tr[5]/td[3]"}
  (Session info: chrome=116.0.5845.111)
Stacktrace:
	GetHandleVerifier [0x00007FF7009752A2+57122]
	(No symbol) [0x00007FF7008EEA92]
	(No symbol) [0x00007FF7007BE3AB]
	(No symbol) [0x00007FF7007F7D3E]
	(No symbol) [0x00007FF7007F7E2C]
	(No symbol) [0x00007FF700830B67]
	(No symbol) [0x00007FF70081701F]
	(No symbol) [0x00007FF70082EB82]
	(No symbol) [0x00007FF700816DB3]
	(No symbol) [0x00007FF7007ED2B1]
	(No symbol) [0x00007FF7007EE494]
	GetHandleVerifier [0x00007FF700C1EF82+2849794]
	GetHandleVerifier [0x00007FF700C71D24+3189156]
	GetHandleVerifier [0x00007FF700C6ACAF+3160367]
	GetHandleVerifier [0x00007FF700A06D06+653702]
	(No symbol) [0x00007FF7008FA208]
	(No symbol) [0x00007FF7008F62C4]
	(No symbol) [0x00007FF7008F63F6]
	(No symbol) [0x00007FF7008E67A3]
	BaseThreadInitThunk [0x00007FFEF4FC26AD+29]
	RtlUserThreadStart

In [41]:
# 기존 station.txt dataframe화 (실행 ㄴㄴ)
sta_df = pd.read_csv('station.txt', sep='\s{1,}', engine='python', names=("KS","STA","LAT","LON"))
sta_df

c:\Users\neimb\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,KS,STA,LAT,LON
0,KS,ADOA,36.572600,128.700900
1,KS,AGSA,37.091700,127.808000
2,KS,AMD,35.343600,126.030000
3,KS,ANDB,33.256580,126.328750
4,KS,ANHA,37.464696,128.155024
...,...,...,...,...
289,KS,YOW2,37.181200,128.456900
290,KS,YPDB,37.608000,125.710000
291,KS,YSAB,36.742000,126.815600
292,KS,YSDA,33.986600,126.920500


In [26]:
# 웹 크롤링으로 가져온 해발고도 dataframe화 (실행 ㄴㄴ)
series = series.to_frame()
series.reset_index(inplace=True)
series.columns=['STA', 'ELEV']

In [40]:
# 두 dataframe 병합 (실행 ㄴㄴ)
data = pd.merge(sta_df, series, how='outer', on='STA')
data

,KS,STA,LAT,LON,ELEV
0,KS,ADOA,36.572600,128.700900,162.0
1,KS,AGSA,37.091700,127.808000,114.0
2,KS,AMD,35.343600,126.030000,79.0
3,KS,ANDB,33.256580,126.328750,84.0
4,KS,ANHA,37.464696,128.155024,540.0
...,...,...,...,...,...
289,KS,YOW2,37.181200,128.456900,287.0
290,KS,YPDB,37.608000,125.710000,97.0
291,KS,YSAB,36.742000,126.815600,85.0
292,KS,YSDA,33.986600,126.920500,57.0


In [45]:
# 위에서 만든 dataframe txt파일로 저장 (실행 ㄴㄴ)
data.to_csv('station.txt', sep=' ', header = None, index = None)

---

In [5]:
# 이전 단계에서 만든 station2.txt를 불러오고 데이터 전처리에 용이하게끔 컬럼명을 추가해줍니다
station = pd.read_csv('station.txt', sep='\s{1,}', engine='python')
station.columns=['NETWORK','STATION','LATITUDE','LONGITUDE','ELEVATION']
station

,NETWORK,STATION,LATITUDE,LONGITUDE,ELEVATION
0,KS,AGSA,37.091700,127.808000,114.0
1,KS,AMD,35.343600,126.030000,79.0
2,KS,ANDB,33.256580,126.328750,84.0
3,KS,ANHA,37.464696,128.155024,540.0
4,KS,ANMA,36.538600,126.330000,89.0
...,...,...,...,...,...
288,KS,YOW2,37.181200,128.456900,287.0
289,KS,YPDB,37.608000,125.710000,97.0
290,KS,YSAB,36.742000,126.815600,85.0
291,KS,YSDA,33.986600,126.920500,57.0


In [21]:
# 위의 datatrame과 sac파일을 기반으로 한 station.sta 자동화

file = os.listdir('../dist100/dist100_station2/')
array = list(filter(lambda x: re.match('.*Z.sac$', x), file))
array.sort(key=lambda x : len(x))
fe = FlinnEngdahl()

with open('station.sta', 'a') as f:
    for i in unit_array:
        st = obspy.read('../dist100/dist100_station2/' + i)
        tnm = st[0].stats.sac.kstnm
        lat, lon, elev = station.loc[station.STATION == tnm].LATITUDE.values[0], station.loc[station.STATION == tnm].LONGITUDE.values[0], station.loc[station.STATION == tnm].ELEVATION.values[0]
        # dataframe의 위도와 경도값을 바탕으로 quadrant값을 받아오고, station.sta에 들어갈 수 있게끔 NS - WE 성분을 쪼개어 포맷팅합니다
        quad_ns, quad_we = fe.get_quadrant(lon, lat).upper()[0], fe.get_quadrant(lon, lat).upper()[1]
        # 계산 및 station.sta 포맷팅에 유용하게끔 위도, 경도값을 정수로 받아주고, 소숫점 아래 *60, 두자리만 출력되게끔 설정해주었습니다 
        int_lat, int_lon = int(lat), int(lon)
        lat, lon = np.round((lat - int_lat)*60,2), np.round((lon - int_lon)*60, 2)
        # 해발고도 값도 포맷팅해줍니다 
        elev = int(elev.round(0))

        # station명이 3글자일경우, 위도/경도/해발고도 값에 예외경우가 있을 시 수정해주는 코드입니다.
        exp1 = re.compile('\d.\d\d')
        exp2 = re.compile('\d\d.\d')
        exp3 = re.compile('\d.\d')
        
        if len(tnm) == 3:
            tnm = ' ' + tnm
        
        if len(str(int_lat)) == 1:
            int_lat = ' ' + str(int_lat)
        if len(str(int_lon)) == 1:
            int_lon = '  ' + str(int_lon)
        elif len(str(int_lon)) == 2:
            int_lon = ' ' + str(int_lon)
            
        if exp1.fullmatch(str(lat)):
            lat = '0' + str(lat)
        elif exp2.fullmatch(str(lat)):
            lat = str(lat) + '0'
        elif exp3.fullmatch(str(lat)):
            lat = '0' + str(lat) + '0'
        
        if exp1.fullmatch(str(lon)):
            lon = '0' + str(lon)
        elif exp2.fullmatch(str(lon)):
            lon = str(lon) + '0'
        elif exp3.fullmatch(str(lon)):
            lon = '0' + str(lon) + '0'
                   
        if len(str(elev)) == 1:
            elev = '   ' + str(elev)
        elif len(str(elev)) == 2:
            elev = '  ' + str(elev)
        elif len(str(elev)) == 3:
            elev = ' ' + str(elev)
        
        line = '{0}{1}{2}{3} {4}{5}{6} {7}\n{0}*'.format(tnm, int_lat, quad_ns, lat, int_lon, quad_we, lon, elev)
        f.write(line + '\n')